![Piggy bank](piggy_bank.jpg)

Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

In [3]:
import pandas as pd
import numpy as np
df = pd.read_csv("bank_marketing.csv")

In [4]:
client_cols = ['client_id', 'age', 'job', 'marital', 'education', 'credit_default','mortgage']
data = [df['client_id'], df['age'], df['job'],df['marital'] ,df['education'] ,df['credit_default'] ,df['mortgage'] ]
client_df = pd.DataFrame(columns = client_cols)

client_df['client_id'] = df['client_id']
client_df['age'] = df['age']
client_df['job'] = df['job']
client_df['marital'] = df['marital']
client_df['education'] = df['education']
client_df['credit_default'] = df['credit_default']
client_df['mortgage'] = df['mortgage']

In [5]:
campaign_cols = ['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date']

campaign_df = pd.DataFrame(columns = campaign_cols)

campaign_df['client_id'] = df['client_id']
campaign_df['number_contacts'] = df['number_contacts']
campaign_df['contact_duration'] = df['contact_duration']
campaign_df['previous_campaign_contacts'] = df['previous_campaign_contacts']
campaign_df['previous_outcome'] = df['previous_outcome']
campaign_df['campaign_outcome'] = df['campaign_outcome']

month_col = df['month'].copy()
month_col.loc[month_col == 'may'] = '5'
month_col.loc[month_col == 'jun'] = '6'
month_col.loc[month_col == 'jul'] = '7'
month_col.loc[month_col == 'oct'] = '10'
month_col.loc[month_col == 'nov'] = '11'
month_col.loc[month_col == 'dec'] = '12'
month_col.loc[month_col == 'mar'] = '3'
month_col.loc[month_col == 'apr'] = '4'
month_col.loc[month_col == 'sep'] = '9'
month_col.loc[month_col == 'aug'] = '8'
campaign_df['last_contact_date'] = '2022' + '-' + month_col + '-' + df['day'].astype(str)
campaign_df['last_contact_date'] = pd.to_datetime(campaign_df['last_contact_date'])

In [6]:
eco_cols = ['client_id', 'cons_price_idx', 'euribor_three_months']
economics_df = pd.DataFrame(columns = eco_cols)
economics_df['client_id'] = df['client_id']
economics_df['cons_price_idx'] = df['cons_price_idx']
economics_df['euribor_three_months'] = df['euribor_three_months']

In [7]:
client_df['credit_default'] = np.where(client_df['credit_default'] == 'yes', '1', '0')
client_df['credit_default'] = client_df['credit_default'].astype(int).astype(bool)

In [8]:
client_df['job'] = client_df['job'].str.replace('.', '-')

In [9]:
client_df['education'] = client_df['education'].str.replace('.', '-')
client_df.loc[client_df['education'] == 'unknown', ['education']] = np.NaN

In [10]:
client_df['mortgage'] = np.where(client_df['mortgage'] == 'yes', '1', '0')
client_df['mortgage'] = client_df['mortgage'].astype(int).astype(bool)

In [11]:
campaign_df['previous_outcome'] = np.where(campaign_df['previous_outcome'] == 'success', '1', '0')
campaign_df['previous_outcome'] = campaign_df['previous_outcome'].astype(int).astype(bool)

In [12]:
campaign_df['campaign_outcome'] = np.where(campaign_df['campaign_outcome'] == 'yes', '1', '0')
campaign_df['campaign_outcome'] = campaign_df['campaign_outcome'].astype(int).astype(bool)

In [13]:
client_df.to_csv('client.csv', index=False)
campaign_df.to_csv('campaign.csv', index=False)
economics_df.to_csv('economics.csv', index=False)